In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import torch
from torch.distributions.normal import Normal

from torch.distributions.multivariate_normal import MultivariateNormal

In [3]:
%matplotlib inline
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
length = 10

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        # Initial State
        self.s_0 = nn.Parameter(torch.tensor([5.525, 5.1, 5.1,5.1], requires_grad=True)).requires_grad_(True)
        
        self.T = nn.Parameter(torch.tensor(torch.rand(4,4), requires_grad=True)).requires_grad_(True)
        self.mu1 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        
        self.mu2 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        
        self.std1 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        self.std2 = nn.Parameter(torch.tensor(torch.rand(4,length), requires_grad=True)).requires_grad_(True)
        

        
        
        

    def forward(self, x,t,pred):

        x = F.softmax((x.clone()),dim=0)
        l=0.0
        std1=torch.exp(self.std1).clip(1e-16,1)
        std2=torch.exp(self.std2).clip(1e-16,1)
        

        x_temp=x.clone()
            
            
            
            
        dist=MultivariateNormal(self.mu1[0]+self.mu2[0]*0.0, (std1[0])*torch.eye(length).to(device))
        
        o_1 = torch.exp(dist.log_prob(pred[0]))

        x_temp[0]=o_1*x[0].clone()

        dist=MultivariateNormal(self.mu1[1]+self.mu2[1]*0.0,( std1[1])*torch.eye(length).to(device))

        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[1]=o_1.clone()*x[1].clone()

        dist=MultivariateNormal(self.mu1[2]+self.mu2[2]*0.0, (std1[2])*torch.eye(length).to(device))
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[2]=o_1*x[2].clone()
        
        dist=MultivariateNormal(self.mu1[3]+self.mu2[3]*0.0, (std1[3])*torch.eye(length).to(device))
        o_1 = torch.exp(dist.log_prob(pred[0]))
        x_temp[3]=o_1*x[3].clone()
        
#         print(x_temp)
        


        x_temp=x_temp.clip(1e-16,1)
        
        x=x_temp.clone()/torch.sum(x_temp.clone())
        l+=torch.log(torch.sum(x_temp.clone()))# -0.01*state
        
        for i in range(1,t): 

            
            x=torch.matmul(x,F.softmax(self.T,dim=1))
            x_temp=x.clone()
            

            x_temp1=x.clone()
            dist=MultivariateNormal(self.mu1[0]+self.mu2[0]*i, (std1[0])*torch.eye(length).to(device))
        
            o_1 = torch.exp(dist.log_prob(pred[i]))

            x_temp[0]=o_1*x[0].clone()

            dist=MultivariateNormal(self.mu1[1]+self.mu2[1]*i, (std1[1])*torch.eye(length).to(device))

            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[1]=o_1.clone()*x[1].clone()

            dist=MultivariateNormal(self.mu1[2]+self.mu2[2]*i,( std1[2])*torch.eye(length).to(device))
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[2]=o_1*x[2].clone()

            dist=MultivariateNormal(self.mu1[3]+self.mu2[3]*i, (std1[3])*torch.eye(length).to(device))
            o_1 = torch.exp(dist.log_prob(pred[i]))
            x_temp[3]=o_1*x[3].clone()

            
            
            
#             print(x_temp)
#             print(pred[i])
            x_temp=x_temp.clip(1e-16,1)

            x=x_temp.clone()/torch.sum(x_temp.clone())
#             print(x)
#             print("-----------------------------")
            l+=torch.log(torch.sum(x_temp.clone()))
        return l
    
    def transition(self,x):
        x=torch.matmul(x.to(device),F.softmax(self.T,dim=1))
        x_temp=x.clone()
        return x


In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)
device='cpu'

cuda:0


In [6]:


T_matrix=torch.rand(3, 3).to(device)
T_matrix=torch.tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]], device=device)

# T_matrix[1,1]=8

# T_matrix[2,2]=8

# T_matrix[0,0]=8

T_matrix=T_matrix/torch.sum(T_matrix, 1).reshape(4,1)


s_0=torch.rand(3).to(device)

s_0=torch.tensor([1,0,0])
s_0=s_0/torch.sum(s_0)

O_matrix_mean=torch.tensor([1.5,2.5,3.5,5.5]).to(device)
O_matrix_mean1=torch.tensor([0.1,0.2,0.3,0.4]).to(device)
O_matrix_std=torch.tensor([1.1,1.2,1.3,1.4]).to(device)






print(T_matrix)

print(s_0)

def simulate_HMM(T_matrix,O_matrix,x,t):
    x=0
    
    
    t=0
    o=[]
#     o_1=torch.matmul(x.to(device),O_matrix.to(device))
# #         print(o_1)
#     o.append(Normal(O_matrix_mean[x],O_matrix_std[x]).sample().to(device))
    
    K=[2*i/(length) for i in range(1,length+1)]
    K=torch.tensor(K).to(device)
    o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+O_matrix_mean1[x]*t*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
    t=1
    for i in range(1,1000):
#         print(i)
        x=(torch.distributions.Categorical(T_matrix[x].to(device)).sample())
#         print(x)
        
        
        t+=1
        if x==0:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+O_matrix_mean1[x]*t*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        if x==1:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+O_matrix_mean1[x]*t*torch.sin(3*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        if x==2:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+O_matrix_mean1[x]*t*torch.sin(6*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        if x==3:
            o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+O_matrix_mean1[x]*t*torch.sin(9*torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )         
        t1=t
        if (t==50) :
            break
    
    

    return o,t,t1



tensor([[0.7000, 0.1300, 0.1000, 0.0700],
        [0.1000, 0.7300, 0.1000, 0.0700],
        [0.2020, 0.2525, 0.4747, 0.0707],
        [0.1000, 0.0700, 0.0400, 0.7900]])
tensor([1., 0., 0.])


In [7]:
o=simulate_HMM(T_matrix,O_matrix_mean,torch.tensor([1.0,0,0.0]),1000)
o

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15832\893764229.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+O_matrix_mean1[x]*t*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15832\893764229.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+O_matrix_mean1[x]*t*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15832\893764229.py:60: UserWarning: 

([tensor([ 2.1640,  0.9936,  2.2386,  4.3022, 10.0460,  6.1868,  1.9354,  1.7265,
           1.2807,  0.5347]),
  tensor([-2.1268,  2.7581, -0.5753,  4.1398, 10.3012,  5.7452,  2.5150,  1.4627,
           0.7123, -0.2646]),
  tensor([2.4099, 1.0931, 1.6975, 4.5488, 9.1959, 3.9743, 3.3460, 3.8993, 1.8104,
          1.3369]),
  tensor([ 2.3058,  2.3927,  2.5375,  5.0896,  9.1525,  5.1324,  4.8050,  2.2888,
           0.9048, -1.9250]),
  tensor([ 1.2196,  0.5995,  3.6313,  5.3297, 10.9328,  6.6280,  4.1134,  2.3997,
           2.4527, -0.2711]),
  tensor([ 1.4953,  0.0717,  2.0842,  5.2165, 11.3498,  5.8821,  2.8818,  2.0222,
          -1.0287,  0.9165]),
  tensor([ 2.2865,  1.0823,  3.0332,  7.4022, 12.6398,  6.9428,  2.4627,  2.3650,
           2.0600,  0.7292]),
  tensor([ 2.0879,  0.1103, -0.6139,  7.9020, 10.6414,  0.9942,  1.5380,  1.5476,
          -2.8609,  1.8867]),
  tensor([ 2.0269, -2.8653,  2.2267,  7.4384, 10.7431,  3.0958,  5.6709,  2.0922,
          -3.1683,  1.1330]),
  

In [8]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import time

start_time = time.time()  
for batch_size in [5]:
    trial =0 
    while trial < 5:
        t=50
        # # batch_size=20
        try: 
            pred1=[]
            for i in range(0,batch_size):
                pred1.append(simulate_HMM(T_matrix,O_matrix_mean,s_0,t))
            import torch.optim as optim
            net = Net()
            net.to(device)
            optimizer = optim.Adam(net.parameters(), lr=.01)
            scheduler =lr_scheduler.LinearLR(optimizer,start_factor=1, end_factor=0.001, total_iters=75)
            net.train()
    
            running_loss = 0.0
            o,t,t__1=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
            # dist=distribution_initalize(o)
            for epoch in range(100):  # loop over the dataset multiple times
    
    
    
    
                # zero the parameter gradients
                optimizer.zero_grad()
    
                # forward + backward + optimize
    
    
    
                loss=0.0
    
                for i in range(0,batch_size):
                    optimizer.zero_grad()
                    loss=0.0
    
            #         o,t=simulate_HMM(T_matrix,O_matrix_mean,s_0,t)
                    o,t,t_11=pred1[i]
                    pred=torch.zeros((1,t), dtype=torch.int32)
                    pred=o
            #         dist=distribution_update(dist,o)
                    loss-=net(net.s_0,t,pred)
    
                    l1_lambda = 0.001  # adjust the L1 regularization parameter as needed
                    l1_reg = torch.tensor(0.).to(device)
                    for param in net.parameters():
                        l1_reg += torch.norm(param, 1).to(device) 
              
                    loss=loss
                    loss.backward()
                    optimizer.step()
                scheduler.step()
                    # print statistics
            # print(f'[{trial + 1}] loss: {loss/t}')
    
            # print('Finished Training')
    
            torch.save(pred1, "data_normal\ " + str(trial)+" _ "+str(batch_size)+ "_data.pt")
            torch.save(net.state_dict(),  "Model_normal\ " + str(trial)+" _ "+str(batch_size)+ "_model.pth")
            trial+=1
        except:
            print(f'[{trial + 1}] loss: {loss/t}')
            
            
            pass
end_time = time.time()  # Capture end time after the loop completes

print(f"Total execution time: {end_time - start_time} seconds")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15832\893764229.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+O_matrix_mean1[x]*t*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15832\893764229.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  o.append(MultivariateNormal(0.1*5/(0.05+(torch.tensor(K)-1)**2)+O_matrix_mean1[x]*t*torch.sin(torch.pi*torch.tensor(K/2)/1),O_matrix_std[x]*torch.eye(length)).sample().to(device)   )
C:\Users\Administrator\AppData\Local\Temp\ipykernel_15832\893764229.py:60: UserWarning: 

Total execution time: 745.372659444809 seconds
